# Chapter 2 : Getting started with the architecture of the transformer model

## **Introduction**

The idea of the attention head of the Transformer is to do away with recurrent neural network features. In this chapter, we will open the hood of the Transformer model described by *Vaswani et al. (2017)* and examine the main components of its architecture. We will explore the fascinating world of attention and illustrate the key components of the Transformer.

This chapter covers the following topics:

* The architecture of the Transformer
* The Transformer’s self-attention model
* The encoding and decoding stacks
* Input and output embedding
* Positional embedding
* Self-attention
* Multi-head attention
* Masked multi-attention
* Residual connections
* Normalization
* Feedforward network
* Output probabilities
Let’s dive directly into the structure of the original Transformer’s architecture.

## **The rise of the Transformer: Attention is All You Need**

The original Transformer model is a stack of $6$ layers. The output of layer **$l$** is the input of layer **$l+1$** until the final prediction is reached. There is a $6$-layer encoder stack on the left and a $6$-layer decoder stack on the right:

![Alt text](transformer_architecture.png)

* On the left, **the inputs enter the encoder side** of the Transformer through **an attention sublayer
*** and **a feedforward sublayer**.
* On the right, **the target outputs go into the decoder side** of the Transformer through **two attention sublayers** and **a feedforward network sublayer**.
  
We immediately notice that there is no RNN, LSTM, or CNN. Recurrence has been abandoned in this architecture.

Attention has replaced recurrence functions requiring increasing parameters as the distance between two words increases. The attention mechanism is a “word to word” operation. It is actually a token-to-token operation, but we will keep it to the word level to keep the explanation simple. The attention mechanism will find how each word is related to all other words in a sequence, including the word being analyzed itself. Let’s examine the following sequence:

```bash
The cat sat on the mat.
```
Attention will run dot products between word vectors and determine the strongest relationships of a word with all the other words, including itself ("cat" and "cat")

![Alt text](attending_to_all_the_words.png)

The attention mechanism will provide **a deeper relationship between words and produce better results**.

For **each attention sublayer**, the original Transformer model runs not one but $8$ attention mechanisms in parallel to speed up the calculations. We will explore this architecture in the following section, *The encoder stack*. This process is named "multi-head attention," providing:
* A broader in-depth analysis of sequences
* The preclusion of recurrence reducing calculation operations
* Implementation of parallelization, which reduces training time
* Each attention mechanism learns different perspectives of the same input sequence

## **The encoder stack**

The layers of the encoder and decoder of the original Transformer model are stacks of layers. Each layer of the encoder stack has the following structure:

![Alt text](layer_of_the_encoder_stack_of_the_transformer.png)

The original encoder layer structure remains the same for all $N=6$ layers of the Transformer model. Each layer contains two main sublayers: <font color=cyan>a multi-headed attention mechanism</font> and <font color=cyan>a fully connected position-wise feedforward network</font>.

Notice that a residual connection surrounds each main sublayer, sublayer(x), in the Transformer model. These connections transport the unprocessed input x of a sublayer to a layer normalization function. This way, we are certain that key information such as positional encoding is not lost on the way. The normalized output of each layer is thus:
$$
LayerNormalization (x + Sublayer(x))
$$

**Though the structure of each of the $N=6$ layers of the encoder is identical, the content of each layer is not strictly identical to the previous layer**.

For example, **the embedding sublayer is only present at the bottom level of the stack**. **The other $5$ layers do not contain an embedding layer**, and **this guarantees that the encoded input is stable through all the layers**.

Also, **the multi-head attention mechanisms perform the same functions from layer $1$ to $6$**. **However, they do not perform the same tasks**. E**ach layer learns from the previous layer and explores different ways of associating the tokens in the sequence**. **It looks for various associations of words**, just like we look for different associations of letters and words when we solve a crossword puzzle.

The designers of the Transformer introduced a very efficient constraint. **The output of every sublayer of the model has a constant dimension, including the embedding layer and the residual connections**. This dimension is $d_{model}$ and can be set to another value depending on your goals. In the original Transformer architecture, $d_{model} = 512$.

$d_{model}$ has a powerful consequence. Practically all the key operations are dot products. As a result, the dimensions remain stable, which reduces the number of operations to calculate, reduces machine consumption, and makes it easier to trace the information as it flows through the model.
This global view of the encoder shows the highly optimized architecture of the Transformer. In the following sections, we will zoom into each of the sublayers and mechanisms.

We will begin with the embedding sublayer.

### **Input embedding**

The **input embedding sublayer converts the input tokens to vectors of dimension $d_{model} = 512$ using learned embeddings in the original Transformer model**. The structure of the input embedding is classical:

![Alt text](input_embedding_sublayer.png)

The embedding sublayer works like other standard transduction models. A tokenizer will transform a sentence into tokens. Each tokenizer has its methods, such as BPE, word piece, and sentence piece methods. The Transformer initially used BPE, but other models use other methods.

The goals are similar, and the choice depends on the strategy chosen. For example, a tokenizer applied to the sequence the Transformer is an innovative NLP model! will produce the following tokens in one type of model:

```bash
['the', 'transform', 'er', 'is', 'an', 'innovative', 'n', 'l', 'p', 'model', '!']
```

You will notice that this tokenizer normalized the string to lowercase and truncated it into subparts. A tokenizer will generally provide an integer representation that will be used for the embedding process. For example:

```Bash
text = "The cat slept on the couch.It was too tired to get up."
tokenized text= [1996, 4937, 7771, 2006, 1996, 6411, 1012, 2009, 2001, 2205, 5458, 2000, 2131, 2039, 1012]
```

There is not enough information in the tokenized text at this point to go further. The tokenized text must be embedded.

The Transformer contains a learned embedding sublayer. Many embedding methods can be applied to the tokenized input.

I chose the skip-gram architecture of the `word2vec` embedding approach Google made available in 2013 to illustrate the embedding sublayer of the Transformer. A skip-gram will focus on a center word in a window of words and predicts context words. For example, if $word(i)$ is the center word in a two-step window, a skip-gram model will analyze $word(i-2)$, $word(i-1)$, $word(i+1)$, and $word(i+2)$. Then the window will slide and repeat the process. A skip-gram model generally contains an input layer, weights, a hidden layer, and an output containing the word embeddings of the tokenized input words.

Suppose we need to perform embedding for the following sentence:

```bash
The black cat sat on the couch and the brown dog slept on the rug.
```

We will focus on two words, ***black*** and ***brown***. The word embedding vectors of these two words should be similar.

Since we must produce a vector of size $d_{model} = 512$ for each word, we will obtain a size $512$ vector embedding for each word:

```bash
black=[[-0.01206071  0.11632373  0.06206119  0.01403395  0.09541149  0.10695464 0.02560172  0.00185677 -0.04284821  0.06146432  0.09466285  0.04642421 0.08680347  0.05684567 -0.00717266 -0.03163519  0.03292002 -0.11397766 0.01304929  0.01964396  0.01902409  0.02831945  0.05870414  0.03390711 -0.06204525  0.06173197 -0.08613958 -0.04654748  0.02728105 -0.07830904
    …
0.04340003 -0.13192849 -0.00945092 -0.00835463 -0.06487109  0.05862355 -0.03407936 -0.00059001 -0.01640179  0.04123065 
-0.04756588  0.08812257 0.00200338 -0.0931043  -0.03507337  0.02153351 -0.02621627 -0.02492662 -0.05771535 -0.01164199 
-0.03879078 -0.05506947  0.01693138 -0.04124579 -0.03779858 
-0.01950983 -0.05398201  0.07582296  0.00038318 -0.04639162 
-0.06819214  0.01366171  0.01411388  0.00853774  0.02183574 
-0.03016279 -0.03184025 -0.04273562]]
```

The word *black* is now represented by $512$ dimensions. Other embedding methods could be used and dmodel could have a higher number of dimensions.

The word embedding of brown is also represented by 512 dimensions:

```bash
brown=[[ 1.35794589e-02 -2.18823571e-02  1.34526128e-02  6.74355254e-02
   1.04376070e-01  1.09921647e-02 -5.46298288e-02 -1.18385479e-02
   4.41223830e-02 -1.84863899e-02 -6.84073642e-02  3.21860164e-02
   4.09143828e-02 -2.74433400e-02 -2.47369967e-02  7.74542615e-02
   9.80964210e-03  2.94299088e-02  2.93895267e-02 -3.29437815e-02
…
  7.20389187e-02  1.57317147e-02 -3.10291946e-02 -5.51304631e-02
  -7.03861639e-02  7.40829483e-02  1.04319192e-02 -2.01565702e-03
   2.43322570e-02  1.92969330e-02  2.57341694e-02 -1.13280728e-01
   8.45847875e-02  4.90090018e-03  5.33546880e-02 -2.31553353e-02
   3.87288055e-05  3.31782512e-02 -4.00604047e-02 -1.02028981e-01
   3.49597558e-02 -1.71501152e-02  3.55573371e-02 -1.77437533e-02
  -5.94457164e-02  2.21221056e-02  9.73121971e-02 -4.90022525e-02]]
```

To verify the word embedding produced for these two words, we can use cosine similarity to see if the word embeddings of the words ***black*** and ***brown*** are similar.

Cosine similarity uses Euclidean ($L2$) norm to create vectors in a unit sphere. The dot product of the vectors we are comparing is the cosine between the points of those two vectors. For more on the theory of cosine similarity, you can consult scikit-learn’s documentation, among many other sources: https://scikit-learn.org/stable/modules/metrics.html#cosine-similarity.

The cosine similarity between the black vector of size $d_{model} = 512$ and the brown vector of size $d_{model} = 512$ in the embedding of the example is:

```Bash
cosine_similarity(black, brown)= [[0.9998901]]
```
The skip-gram produced two vectors that are close to each other. It detected that black and brown form a color subset of the dictionary of words.

The Transformer’s subsequent layers do not start empty-handed. They have learned word embeddings that already provide information on how the words can be associated.

However, a big chunk of information is missing because no additional vector or information indicates a word’s position in a sequence.

The designers of the Transformer came up with yet another innovative feature: positional encoding.

Let’s see how positional encoding works.

### **Positional encoding**

We **enter this positional encoding function of the Transformer with no idea of the position of a word in a sequence**:

![Alt text](positional_encoding.png)

We cannot create independent positional vectors that would have a high cost on the training speed of the Transformer and make attention sublayers overly complex to work with. **The idea is to add a positional encoding value to the input embedding instead of having additional vectors to describe the position of a token in a sequence**.

> Industry 4.0 is pragmatic and model-agnostic. **The original Transformer model has only one vector that contains word embedding and position encoding**. We will explore disentangled attention with a separate matrix for positional encoding in Chapter 15, From NLP to Task-Agnostic Transformer Models.

The Transformer expects a fixed size $d_{model} = 512$ (or other constant value for the model) for each vector of the output of the positional encoding function.

If we go back to the sentence we used in the word embedding sublayer, we can see that black and brown may be semantically similar, but they are far apart in the sentence:

```bash
The *black* cat sat on the couch and the *brown* dog slept on the rug.
```
The word ***black*** is in position 2, `pos=2`, and the word ***brown*** is in position 10, `pos=10`.

Our problem is to find a way to add a value to the word embedding of each word so that it has that information. However, we need to add a value to the $d_{model} = 512$ dimensions! For each word embedding vector, we need to find a way to provide information to `i` in the `range(0,512)` dimensions of the word embedding vector of ***black*** and ***brown***.

***There are many ways to achieve positional encoding***. This section will focus on the designers’ clever way to use a unit sphere to represent positional encoding with sine and cosine values that will thus remain small but useful.

Vaswani et al. (2017) provide sine and cosine functions so that we can generate different frequencies for the positional encoding (**PE**) for each position and each dimension i of the $d_{model} = 512$ of the word embedding vector:

$$
PE_{(pos~2i)} = sin \left( \frac{pos}{10000\frac{2i}{d_model}} \right)
$$

$$
PE_{(pos~2i + 1)} = cos \left( \frac{pos}{10000\frac{2i}{d_model}} \right)
$$

If we start at the beginning of the word embedding vector, we will begin with a constant $(512)$, $i=0$, and end with $i=511$. This means that the sine function will be applied to the even numbers and the cosine function to the odd numbers. Some implementations do it differently. In that case, the domain of the sine function can be $i \in [0, 255]$ and the domain of the cosine function can be $i \in [256, 512]$. This will produce similar results.

In this section, we will use the functions the way they were described by Vaswani et al. (2017). A literal translation into Python pseudo code produces the following code for a positional vector `pe[0][i]` for a position `pos`:

```py
def positional_encoding(pos, pe):
    for i in range(0, 512,2):
         pe[0][i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
         pe[0][i+1] = math.cos(pos / (10000 ** ((2 * i)/d_model)))
    return pe
```

Before going further, you might want to see the plot of the sine function, for example, for `pos=2`.

You can Google the following plot, for example:

```py
plot y=sin(2/10000^(2*x/512))
```

You will obtain the following graph:

![Alt text](graph.png)

If we go back to the sentence we are parsing in this section, we can see that black is in position pos=2 and brown is in position pos=10:

```bash
The black cat sat on the couch and the brown dog slept on the rug.
```
If we apply the sine and cosine functions literally for `pos=2`, we obtain a `size=512` positional encoding vector:

```bash
PE(2)= 
[[ 9.09297407e-01 -4.16146845e-01  9.58144367e-01 -2.86285430e-01
   9.87046242e-01 -1.60435960e-01  9.99164224e-01 -4.08766568e-02
   9.97479975e-01  7.09482506e-02  9.84703004e-01  1.74241230e-01
   9.63226616e-01  2.68690288e-01  9.35118318e-01  3.54335666e-01
   9.02130723e-01  4.31462824e-01  8.65725577e-01  5.00518918e-01
   8.27103794e-01  5.62049210e-01  7.87237823e-01  6.16649508e-01
   7.46903539e-01  6.64932430e-01  7.06710517e-01  7.07502782e-01
…
   5.47683925e-08  1.00000000e+00  5.09659337e-08  1.00000000e+00
   4.74274735e-08  1.00000000e+00  4.41346799e-08  1.00000000e+00
   4.10704999e-08  1.00000000e+00  3.82190599e-08  1.00000000e+00
   3.55655878e-08  1.00000000e+00  3.30963417e-08  1.00000000e+00
   3.07985317e-08  1.00000000e+00  2.86602511e-08  1.00000000e+00
   2.66704294e-08  1.00000000e+00  2.48187551e-08  1.00000000e+00
   2.30956392e-08  1.00000000e+00  2.14921574e-08  1.00000000e+00]]
```

We also obtain a `size=512` positional encoding vector for position 10, `pos=10`:

```bash
PE(10)= 
[[-5.44021130e-01 -8.39071512e-01  1.18776485e-01 -9.92920995e-01
   6.92634165e-01 -7.21289039e-01  9.79174793e-01 -2.03019097e-01
   9.37632740e-01  3.47627431e-01  6.40478015e-01  7.67976522e-01
   2.09077001e-01  9.77899194e-01 -2.37917677e-01  9.71285343e-01
  -6.12936735e-01  7.90131986e-01 -8.67519796e-01  4.97402608e-01
  -9.87655997e-01  1.56638563e-01 -9.83699203e-01 -1.79821849e-01
…
  2.73841977e-07  1.00000000e+00  2.54829672e-07  1.00000000e+00
   2.37137371e-07  1.00000000e+00  2.20673414e-07  1.00000000e+00
   2.05352507e-07  1.00000000e+00  1.91095296e-07  1.00000000e+00
   1.77827943e-07  1.00000000e+00  1.65481708e-07  1.00000000e+00
   1.53992659e-07  1.00000000e+00  1.43301250e-07  1.00000000e+00
   1.33352145e-07  1.00000000e+00  1.24093773e-07  1.00000000e+00
   1.15478201e-07  1.00000000e+00  1.07460785e-07  1.00000000e+00]]
```

When we look at the results we obtained with an intuitive literal translation of the Vaswani et al. (2017) functions into Python, we would like to check whether the results are meaningful.

The cosine similarity function used for word embedding comes in handy for having a better visualization of the proximity of the positions:

```bash
cosine_similarity(pos(2), pos(10))= [[0.8600013]]
```
The similarity between the position of the words black and brown and the lexical field (groups of words that go together) similarity is different:
```bash
cosine_similarity(black, brown)= [[0.9998901]]
```
The encoding of the position shows a lower similarity value than the word embedding similarity.

The positional encoding has taken these words apart. Bear in mind that word embeddings will vary with the corpus used to train them. The problem is now how to add the positional encoding to the word embedding vectors.

### **Adding positional encoding to the embedding vector**

The authors of the Transformer found a simple way by merely adding the positional encoding vector to the word embedding vector:

![Alt text](positional_encoding_addition.png)

If we go back and take the word embedding of **black**, for example, and name it $y1 = black$, we are ready to add it to the positional vector pe(2) we obtained with positional encoding functions. We will obtain the positional encoding pc(black) of the input word **black**:
$$
pc(black) = y_1 + pe(2)
$$

The solution is straightforward. However, if we apply it as shown, we might lose the information of the word embedding, which will be minimized by the positional encoding vector.

There are many possibilities to increase the value of y1 to make sure that the information of the word embedding layer can be used efficiently in the subsequent layers.

One of the many possibilities is to add an arbitrary value to y1, the word embedding of **black**:
$$
y_1 * math.sqrt(d_{model})
$$

We can now add the positional vector to the embedding vector of the word black, which are both of the same size ($512$):

```python
for i in range(0, 512,2):
        pe[0][i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
        pc[0][i] = (y[0][i]*math.sqrt(d_model))+ pe[0][i]
        
        pe[0][i+1] = math.cos(pos / (10000 ** ((2 * i)/d_model)))
        pc[0][i+1] = (y[0][i+1]*math.sqrt(d_model))+ pe[0][i+1]
```

The result obtained is the final positional encoding vector of dimension dmodel = 512:
```bash
pc(black)=
[[ 9.09297407e-01 -4.16146845e-01  9.58144367e-01 -2.86285430e-01
   9.87046242e-01 -1.60435960e-01  9.99164224e-01 -4.08766568e-02
   …
  4.74274735e-08  1.00000000e+00  4.41346799e-08  1.00000000e+00
   4.10704999e-08  1.00000000e+00  3.82190599e-08  1.00000000e+00
   2.66704294e-08  1.00000000e+00  2.48187551e-08  1.00000000e+00
   2.30956392e-08  1.00000000e+00  2.14921574e-08  1.00000000e+00]]
```
The same operation is applied to the word ***brown*** and all of the other words in a sequence.

We can apply the cosine similarity function to the positional encoding vectors of ***black*** and ***brown***:
```bash
cosine_similarity(pc(black), pc(brown))= [[0.9627094]]
```

We now have a clear view of the positional encoding process through the three cosine similarity functions we applied to the three states representing the words black and brown:
```bash
[[0.99987495]] word similarity
[[0.8600013]] positional encoding vector similarity
[[0.9627094]] final positional encoding similarity
```
We saw that the initial word similarity of their embeddings was high, with a value of $0.99$. Then we saw the positional encoding vector of positions 2 and 10 drew these two words apart with a lower similarity value of $0.86$.

Finally, we added the word embedding vector of each word to its respective positional encoding vector. We saw that this brought the cosine similarity of the two words to $0.96$.

The positional encoding of each word now contains the initial word embedding information and the positional encoding values.

The output of positional encoding leads to the multi-head attention sublayer.

### **Sublayer 1 : Multi-head attention**

The multi-head attention sublayer **contains $8$ heads and is followed by post-layer normalization**, which will **add residual connections to the output of the sublayer and normalize it**:

![Alt text](multi_head_attention_sublayer.png)

This section begins with the architecture of an attention layer. Then, an example of multi-attention is implemented in a small module in Python. Finally, post-layer normalization is described.

Let’s start with the architecture of multi-head attention.

#### **The architecture of multi-head attention**

The input of the multi-attention sublayer of the first layer of the encoder stack is a vector that contains the embedding and the positional encoding of each word. The next layers of the stack do not start these operations over.

The dimension of the vector of each word xn of an input sequence is $d_{model} = 512$:

$$
pe(x_n)=[d_1=9.09297407e-01, d_2=-4.16146845e-01, \dots, d_{512}=1.00000000e+00]
$$

The representation of each word xn has become a vector of $d_{model} = 512$ dimensions.
Each word is mapped to all the other words to determine how it fits in a sequence.
In the following sentence, we can see that it could be related to **cat** and **rug** in the sequence:
```bash
Sequence = The cat sat on the rug and it was dry-cleaned.
```

The model will train to find out if **it** is related to **cat** or **rug**. We could run a huge calculation by training the model using the $d_{model} = 512$ dimensions as they are now.

However, we would only get one point of view at a time by analyzing the sequence with one dmodel block. Furthermore, it would take quite some calculation time to find other perspectives.

A better way is to divide the $d_{model} = 512$ dimensions of each word $x_n$ of $x$ (all the words of a sequence) into $8$ $d_k=64$ dimensions. We then can run the 8 “heads” in parallel to speed up the training and obtain 8 different representation subspaces of how each word relates to another:

![Alt text](multi_head_representation.png)

You can see that there are now **8** heads running in parallel. One head might decide that **it** fits well with **cat** and another that **it** fits well with **rug** and another that **rug** fits well with **dry-cleaned**.

The output of each head is a matrix Zi with a shape of x * dk. The output of a multi-attention head is Z defined as:

$$
Z = (Z_0, Z_1, Z_2, Z_3, Z_4, Z_5, Z_6, Z_7)
$$

However, $Z$ must be concatenated so that the output of the multi-head sublayer is not a sequence of dimensions but one line of an $xm * d_{model}$ matrix.

Before exiting the multi-head attention sublayer, the elements of Z are concatenated:

$$
MultiHead(output) = Concat(Z_0, Z_1, Z_2, Z_3, Z_4, Z_5, Z_6, Z_7) = x, d_{model}
$$

Notice that each head is concatenated into $z$ that has a dimension of $d_{model} = 512$. The output of the multi-headed layer respects the constraint of the original Transformer model.

Inside each head $h_n$ of the attention mechanism, the "word" matrices have three representations:

* A query matrix ($Q$) that has a dimension of $d_q = 64$, which seeks all the key-value pairs of the "word" matrices.
* A key matrix ($K$) that has a dimension of $d_k = 64$, which will be trained to provide an attention value.
* A value matrix ($V$) that has a dimension of $d_v = 64$, which will be trained to provide another attention value.

Attention is defined as **"Scaled Dot-Product Attention,"** which is represented in the following equation in which we plug $Q$, $K$, and $V$:

$$
Attention(Q, K, V) = softmax\left( \frac{QK^T}{\sqrt{d_k}} \right)V
$$

The matrices all have the same dimension, making it relatively simple to use a scaled dot product to obtain the attention values for each head and then concatenate the output Z of the 8 heads.

To obtain Q, K, and V, we must train the model with their weight matrices $Q_w$, $K_w$, and $V_w$, which have $d_k = 64$ columns and $d_{model} = 512$ rows. For example, $Q$ is obtained by a dot-product between x and Qw. Q will have a dimension of $d_k = 64$.

> ou can modify all the parameters, such as the number of layers, heads, $d_{model}$, $d_k$, and other variables of the Transformer to fit your model. This chapter describes the original Transformer parameters by Vaswani et al. (2017). It is essential to understand the original architecture before modifying it or exploring variants of the original model designed by others.

Google Brain Trax, OpenAI, and Hugging Face, among others, provide ready-to-use libraries that we will be using throughout this book. However, let’s open the hood of the Transformer model and get our hands dirty in Python to illustrate the architecture we just explored to visualize the model in code and show it with intermediate images.

We will use basic Python code with only **numpy** and a **softmax** function in 10 steps to run the key aspects of the attention mechanism.

> Bear in mind that an Industry 4.0 developer will face the challenge of multiple architectures for the same algorithm.

See the `Multi_Head_Attention_Sub_Layer.ipynb` file

### **Post-layer normalization**

Each attention sublayer and each Feedforward sublayer of the Transformer is followed by **post-layer normalization (Post-LN)**:

![Alt text](post_layer_normalization.png)

The Post-LN contains an add function and a layer normalization process. The add function processes the residual connections that come from the input of the sublayer. The goal of the residual connections is to make sure critical information is not lost. The Post-LN or layer normalization can thus be described as follows:

$$
LayerNormalization (x + Sublayer(x))
$$

$Sublayer(x)$ is the sublayer itself. $x$ is the information available at the input step of $Sublayer(x)$.

The input of the $LayerNormalization$ is a vector $v$ resulting from $x + Sublayer(x)$. $d_{model} = 512$ for every input and output of the Transformer, which standardizes all the processes.

Many layer normalization methods exist, and variations exist from one model to another. The basic concept for $v = x + Sublayer(x)$ can be defined by $LayerNormalization(v)$:

$$
LayerNormalization(v) = \gamma \frac{v-\mu}{\sigma} + \beta
$$

The variables are:

* $\mu$ is the mean of $v$ of dimension $d$. As such:

$$
\mu = \frac{1}{d}\sum^{d}_{k=1}v_k
$$
* $\sigma$ is the standard deviation $v$ of dimension $d$. As such:

$$
\sigma^2 = \frac{1}{d}\sum^{d}_{k=1}(v_{k-\mu})
$$

* $\gamma$ is a scaling parameter.
* $\beta$ is a bias vector.

This version of $LayerNormalization (v)$ shows the general idea of the many possible post-LN methods. The next sublayer can now process the output of the post-LN or $LayerNormalization (v)$. In this case, the sublayer is a feedforward network.

### **Sublayer 2 : Feedforward network**

The input of the **feedforward network (FFN)** is the $d_{model} = 512$ output of the post-LN of the previous sublayer:

![Alt text](feedforward_layer.png)

The FFN sublayer can be described as follows:

* The FFNs in the encoder and decoder are fully connected.
* The FFN is a position-wise network. Each position is processed separately and in an identical way.
* The FFN contains two layers and applies a ReLU activation function.
* The input and output of the FFN layers is $d_{model} = 512$, but the inner layer is larger with $d_{ff} = 2048$.
* The FFN can be viewed as performing two convolutions with size 1 kernels.
Taking this description into account, we can describe the optimized and standardized FFN as follows:
$$
FFN(x) = max (0, xW_1 + b_1) W_2+b_2
$$
The output of the FFN goes to post-LN, as described in the previous section. Then the output is sent to the next layer of the encoder stack and the multi-head attention layer of the decoder stack.

Let’s now explore the decoder stack.

## **The decoder stack**

The layers of the decoder of the Transformer model are stacks of layers like the encoder layers. Each layer of the decoder stack has the following structure:

![Alt text](a_layer_of_decoder_stack_of_transformer.png)

**The structure of the decoder layer remains the same as the encoder for all the $N=6$ layers of the Transformer model**. Each layer contains $3$ sublayers: **a multi-headed masked attention mechanism**, **a multi-headed attention mechanism**, and a **fully connected position-wise feedforward network**.

The decoder has a third main sublayer, which is **the masked multi-head attention mechanism**. In this sublayer output, **at a given position, the following words are masked so that the Transformer bases its assumptions on its inferences without seeing the rest of the sequence**. That way, in this model, it cannot see future parts of the sequence.

A residual connection, $Sublayer(x)$, surrounds each of the three main sublayers in the Transformer model like in the encoder stack:
$$
LayerNormalization (x + Sublayer(x))
$$

The embedding layer sublayer is only present at the bottom level of the stack, like for the encoder stack. The output of every sublayer of the decoder stack has a constant dimension, $d_{model}$ like in the encoder stack, including the embedding layer and the output of the residual connections.

We can see that the designers worked hard to create symmetrical encoder and decoder stacks.

The structure of each sublayer and function of the decoder is similar to the encoder. In this section, we can refer to the encoder for the same functionality when we need to. We will only focus on the differences between the decoder and the encoder.

### **Output embedding and position encoding**

The structure of the sublayers of the decoder is mostly the same as the sublayers of the encoder. The output embedding layer and position encoding function are the same as in the encoder stack.

In the Transformer usage we are exploring through the model presented by Vaswani et al. (2017), the output is a translation we need to learn. I chose to use a French translation:

```bash
Output=Le chat noir était assis sur le canapé et le chien marron dormait sur le tapis 
```

This output is the French translation of the English input sentence:

```bash
Input=The black cat sat on the couch and the brown dog slept on the rug.
```

The output words go through the word embedding layer and then the positional encoding function like in the first layer of the encoder stack.

Let’s see the specific properties of the multi-head attention layers of the decoder stack.

### **The attention layers**

The Transformer is **an auto-regressive model**. It **uses the previous output sequences as an additional input**. **The multi-head attention layers of the decoder use the same process as the encoder**.

However, **the masked multi-head attention sublayer 1 only lets attention apply to the positions up to and including the current position**. The future words are **hidden from the Transformer**, and this **forces it to learn how to predict**.

A **post-layer normalization process** follows the masked multi-head attention sublayer 1 as in the encoder.

The multi-head attention sublayer 2 also only attends to the positions up to the current position the Transformer is predicting to avoid seeing the sequence it must predict.

The multi-head attention sublayer 2 draws information from the encoder by taking encoder $(K, V)$ into account during the dot-product attention operations. This sublayer also draws information from the masked multi-head attention sublayer 1 (masked attention) by also taking sublayer 1$(Q)$ into account during the dot-product attention operations. The decoder thus uses the trained information of the encoder. We can define the input of the self-attention multi-head sublayer of a decoder as:

$$
Input_Attention = (Output_decoder_sub_layer – 1(Q), Output_encoder_layer(K, V))
$$

A **post-layer normalization process** follows the masked multi-head attention sublayer 1 as in the encoder.

The Transformer then goes to the FFN sublayer, followed by a post-LN and the linear layer.

### **The FFN sublayer, the post-LN and the linear layer**

The FFN sublayer has the same structure as the FFN of the encoder stack. The post-layer normalization of the FFN works as the layer normalization of the encoder stack.

The Transformer produces an output sequence of only one element at a time:
$$
Output~sequence = (y_1, y_2, \dots y_n)
$$
The linear layer produces an output sequence with a linear function that varies per model but relies on the standard method:

$$
y = w * x + b
$$
$w$ and $b$ are learned parameters.

The linear layer will thus produce the next probable elements of a sequence that a softmax function will convert into a probable element.

The decoder layer, like the encoder layer, will then go from layer $l$ to layer $l+1$, up to the top layer of the $N=6$-layer transformer stack.

Let’s now see how the Transformer was trained and the performance it obtained.

## **Training and performance**

The original Transformer was trained on a $4.5$ million sentence pair English-German dataset and a 36 million sentence pair English-French dataset.

The datasets come from Workshops on Machine Translation (WMT), which can be found at the following link if you wish to explore the WMT datasets: http://www.statmt.org/wmt14/

The training of the original Transformer base models took $12$ hours to train for $100,000$ steps on a machine with $8$ NVIDIA $P100$ GPUs. The big models took $3.5$ days for $300,000$ steps.

The original Transformer outperformed all the previous machine translation models with a BLEU score of 41.8. The result was obtained on the WMT English-to-French dataset.

BLEU stands for Bilingual Evaluation Understudy. It is an algorithm that evaluates the quality of the results of machine translations.

The Google Research and Google Brain team applied optimization strategies to improve the performance of the Transformer. For example, the Adam optimizer was used, but the learning rate varied by first going through warmup states with a linear rate and decreasing the rate afterward.

Different types of regularization techniques, such as residual dropout and dropouts, were applied to the sums of embeddings. Also, the Transformer applies label smoothing that avoids overfitting with overconfident one-hot outputs. It introduces less accurate evaluations and forces the model to train more and better.

Several other Transformer model variations have led to other models and usages that we will explore in the subsequent chapters.

Before end the chapter, let’s get a feel of the simplicity of ready-to-use transformer models in Hugging Face, for example.

## **Tranformer models in Hugging Face**

**Everything you saw in this chapter can be condensed in to a ready-to-use Hugging Face transformer model**. With Hugging Face, you can implement machine translation in three lines of code!

Open `Multi_Head_Attention_Sub_Layer.ipynb` in Google Colaboratory. Save the notebook in your Google Drive (make sure you have a Gmail account). Go to the two last cells.